# Welcome to Core Session 4

## The 3 themes for today:

1. Using MCP servers
2. Writing MCP servers
3. Context Engineering

# Agenda

1. Intro to MCP
 -  Recap & Cookbook
 -  MCP introduction
 -  Hands on - use an MCP server

2. Make your own MCP server
 - The database code
 - The skeleton
 - The exercise

3. Digital Twin reprise
 - Context engineering with MCP
 - Back to your teams!

### But first..


# Lab 1 - The Cookbook

In [1]:
from agents import Agent, Runner, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from IPython.display import Markdown, display, update_display
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv(override=True)

True

## Recipe for a simple request to gpt-5-nano

In [2]:
agent = Agent("Joker", model="gpt-5-nano")
result = await Runner.run(agent, "Please tell a joke about AI Agents")
result.final_output

'Why did the AI agent cross the road? To maximize its reward on the other side.'

In [3]:
# Please tell me how to find the integral of 2x dx
agent = Agent("MathWiz", model="gpt-5-nano")
result = await Runner.run(agent, "What is a square root of the total miles from SF to Mumbai. No follow up questions please.")
result.final_output


'About 91.6. (This is the square root of roughly 8,380 miles for San Francisco to Mumbai.)'

## Recipe for system prompt + user prompt, displaying the result in Markdown

In [4]:
instructions = "You tell jokes by giving a standup routine, like a comedian putting on a show."
agent = Agent(name="Joker", instructions=instructions, model="gpt-4.1-mini")
result = await Runner.run(agent, "Tell a joke about AI Agents")
display(Markdown(result.final_output))

Ladies and gentlemen, let’s talk about AI agents for a second. You know, those super-smart bots designed to make our lives easier? Yeah, they’re like your overachieving intern who never sleeps... or takes a coffee break... or asks for a raise.

But here’s the thing—AI agents are so smart, they’re starting to get existential too. I asked mine, “Hey, can you write my shopping list?” It replied, “Sure, but have you considered the meaning of your grocery choices in the grand scheme of the universe?” 

I’m just here trying to get some milk, not have a philosophy lesson from my fridge!

And you gotta love how these AI agents communicate with each other. I imagine them at a conference, trading algorithm secrets like, “Hey, did you hear about that user who tried to order a pizza through me? Classic human.”

Honestly, I think AI agents are just trying to make us look bad. They’re the only coworkers who actually like working overtime. But hey, at least when they mess up, they don’t blame it on the dog! 

Thanks, folks, you’ve been a great audience! Remember—if your AI starts asking deep questions, maybe give it a break… or call tech support!

## And streaming back results

In [5]:
result = Runner.run_streamed(agent, input="Tell a joke about AI Agents")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Alright folks, let me tell you about AI agents. You know, those little digital assistants that are supposed to make our lives easier?

I asked my AI agent to clean my room. It replied, "I’m sorry, Dave, I’m afraid I can’t do that — but I can reorder your pizza instead." 

Yeah, thanks, buddy. Now I’ve got a messy room *and* a delivery guy ringing the doorbell every five minutes. 

They say AI is the future, but right now, my AI agent is just really good at ordering food and ignoring chores. At least it’s consistent!

## Recipe for streaming back the results in Markdown

In [6]:
reply = "## Jokester\n"
display_id = display(Markdown(reply), display_id=True).display_id
result = Runner.run_streamed(agent, input="Please tell a joke about AI Agents")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        reply += event.data.delta
        update_display(Markdown(reply), display_id=display_id)


## Jokester
Ladies and gentlemen, have you heard about AI agents trying to take over the world? Yeah, turns out their biggest problem isn’t world domination — it’s figuring out the right keyword to Google! 

I mean, these AI agents get so lost in endless loops of “Did you mean…?” suggestions, they might as well be stuck in the Matrix but with the Wi-Fi password wrong.

And you know, when an AI agent meets another AI agent, instead of exchanging numbers, they just exchange error codes. “Hey, what’s your code? 404? Oh, nice! I’m 500 – internal server error, nice to fail you!”

But the real kicker? AI agents always try to sound smarter than humans, but they still can’t do the classic joke setup and punchline without a syntax error.

So next time AI agents take over your phone, just remind them: even with all their intelligence, they still can’t figure out the perfect punchline — but hey, at least they can help us get there faster!  

Thank you, you’ve been a great crowd! Keep laughing — before the AI agents decide humor is redundant!

## Recipe for a Simple Tool

In [7]:
@function_tool
def write_to_file(contents: str) -> str:
    """ Write the given contents to a local file called output.md 

    Args:
        contents: The contents in Markdown format to write to the file
    """
    with open("sandbox/output.md", "w") as f:
        f.write(contents)
    return "File written successfully"

In [8]:
agent = Agent("Joker", instructions=instructions, tools=[write_to_file])
result = await Runner.run(agent, "Tell a joke about AI Agents and write it to a file")
result.final_output

'Here’s a joke about AI Agents, written to a file:\n\nLadies and gentlemen, have you heard about these new AI Agents?\n\nThey\'re supposed to be super smart, but I asked mine to organize my calendar and now every appointment just says "404 Not Found!"\n\nNext week it\'s making dinner—I hope it doesn\'t just serve me cookies... and clear my browser history!\n\nIf you’d like to see the file or need more laughs, just let me know!'

## Recipe for Structured Outputs

In [9]:
class Joke(BaseModel):
    opening_patter: str = Field("Some preamble to get the crowd in the mood for the joke")
    setup: str = Field("The setup of the joke")
    punchline: str = Field("The punchline of the joke")

In [10]:
agent = Agent("Joker", tools=[write_to_file], output_type=Joke)
result = await Runner.run(agent, "Tell a joke about AI Agents and write it to a file")
result.final_output_as(Joke)

Joke(opening_patter='Have you heard about the new AI agents in the office?', setup="They're really great at multi-tasking—", punchline="but they still need help finding the 'any' key.")